In [ ]:
import json
import numpy as np
import os
import time
import cv2

In [ ]:
def load_video_as_frames(filename):
    frames=[]
    
    video = cv2.VideoCapture(filename)
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fps = video.get(cv2.CAP_PROP_FPS)
    
    index = 1
    thr = 90
    num_section = 10
    len_section = np.floor(size[0]/ num_section).astype(int)
    success = True

    while success:
        
        success, frame = video.read()
        if not success:
            break
        index += 1
        frames.append(frame[:, :, 0])

    video.release()
    cv2.destroyAllWindows()
    
    return frames,fps,size
    
    

In [ ]:
filename="010_RI_full.mp4"

In [ ]:
frames,fps,size = load_video_as_frames(filename)

In [ ]:
def show_frames(video,fps=30):
    delay = int(1000 / fps)
    for img in video:
        cv2.imshow("new video", img)
        if cv2.waitKey(delay) == 27:
            break
    cv2.destroyAllWindows()
    return True

In [ ]:
show_frames(frames,fps)

In [ ]:
def empiric_confidence(x):
    # x,column_id
    confidence_list=[0.7,0.9,1,1,1,1,1,1,0.9,0.7]
    return confidence_list[x]

In [ ]:

def init_points(video,size=None,num_section=10,thr=90):
    
    if not size:
        size=[video[0].shape[1],video[0].shape[0]]
    
    len_section = np.floor(size[0]/ num_section).astype(int)
    
    points_video = []
    
    for frame_id in range(len(video)):
        wst, frame = cv2.threshold(video[frame_id], thr, 255, 0)
        points_frame = []
        for section_id in range(num_section):
            tmp=frame[:,section_id*len_section:(section_id+1)*len_section]
            total = sum([sum(i) for i in tmp])
            count=0
            for idx,lice in enumerate(tmp):
                count+=sum(lice)
                if count>=total/2:
                    if True:
                        points_frame.append([(section_id+0.5)*len_section,idx,empiric_confidence(section_id),frame_id])
                    break
        points_video.append(points_frame.copy())
        
    return points_video

In [ ]:
points_video = init_points(frames)

In [ ]:
def put_points(video,points_video,fps):
    
    delay = int(1000 / fps)
    imgs=[cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in video]
    
    for img_id in range(len(imgs)):
        for x in range(10):
            cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id][x][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
    
    return imgs
    

In [ ]:
def put_points(video,points_video,fps):
    
    delay = int(1000 / fps)
    imgs=[cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in video]
    
    for img_id in range(len(imgs)):
        for x in range(10):
            cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id][x][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1)
            if 0<img_id<len(imgs)-1:
                if x in [0,1]:
                    cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id+1][x+2][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
                if x in [8,9]:
                    cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id-1][x-2][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
    return imgs

In [ ]:
raw_eval=put_points(frames,points_video,fps)

In [ ]:
show_frames(raw_eval,fps)

In [ ]:
size

In [ ]:
def save_video(filename,frames,fps,size):
    
    videoWriter = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
    
    for frame in frames:
        videoWriter.write(frame)
        
    videoWriter.release()

In [ ]:
save_video("show.mp4",raw_eval,fps,size)

In [ ]:
def load_templates(filepath="template/LAS/ribs/"):
    templates = []
    
    for filename in os.listdir(filepath):
        org = cv2.imread(filepath+filename,0)
        templates.append(org.copy())
        
    return templates

In [ ]:
templates = load_templates("template/LAS/ribs/")

In [ ]:
len(templates)

In [ ]:
def template_matching(img,templates,thr=0.8):
    
    h,w = [template.shape[0] for template in templates], [template.shape[1] for template in templates]
    
    res_list=[]

    VL=[]
    for i in range(len(templates)):
        res = cv2.matchTemplate(img, templates[i], cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        VL.append([max_val,max_loc,i])
    VL.sort(key=lambda x:-x[0])

    for i in range(3):
        if VL[i][0]>thr:
            top_left = VL[i][1]
            bottom_right = (top_left[0] + w[VL[i][2]], top_left[1] + h[VL[i][2]])
            res_list.append([top_left,bottom_right,VL[i][2],VL[i][0]])
    
    return  res_list      #[coordinates(top_left,bottom_right),template_id,NCC_value(=confidence_value)]

In [ ]:
ribs_info = template_matching(frames[70],templates)

In [ ]:
ribs_info

In [ ]:
a,b,c,d=ribs_info[0]

In [ ]:
def draw_templates(frame,info,thr=0):
    img=frame.copy()
    for temp in info:
        top_left,bottom_right,temp_id,confidence,frame_id=temp
        if confidence>thr:
            cv2.rectangle(img, top_left, bottom_right, 255, 1)
            cv2.putText(img,"template:"+str(temp_id)+"  Confidence:"+str(confidence),(top_left[0],top_left[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 1)
    return img
    

In [ ]:
img = draw_templates(frames[70],ribs_info)

In [ ]:
cv2.imshow("t",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def get_video_info(video,templates):
    video_info=[]
    
    for i in range(len(video)):
        frame_infos = template_matching(video[i],templates)
        L=[]
        if frame_infos:
            for frame_info in frame_infos:
                top_left,bottom_right,temp_id,confidence=frame_info
                L.append([top_left,bottom_right,temp_id,confidence,i])
        if L:
            video_info.append(L.copy())
    return video_info

In [ ]:
video_info = get_video_info(frames,templates)

# Ribs position analysis

In [ ]:
#compute centroids
video_info_cen=[]
for frame_info in video_info:
    for piece in frame_info:
        top_left,bottom_right,temp_id,confidence,frame_id=piece
        centroid=[np.mean([top_left[i],bottom_right[i]]).astype(int) for i in range(len(top_left))]
        video_info_cen.append([centroid,temp_id,confidence,frame_id])
        

In [ ]:
y_info=[i[0][1] for i in video_info_cen]
y_median = np.median(y_info).astype(int)
y_mean = np.mean(y_info).astype(int)

In [ ]:
#compute centroids
Bottom=[]
for frame_info in video_info:
    for piece in frame_info:
        top_left,bottom_right,temp_id,confidence,frame_id=piece
        Bottom.append(bottom_right)

In [ ]:
y_info=[i[1] for i in Bottom]
y_median = np.median(y_info).astype(int)
y_mean = np.mean(y_info).astype(int)

In [ ]:
def median_bottom(info):
    Bottom=[]
    for frame_info in info:
        for piece in frame_info:
            top_left,bottom_right,temp_id,confidence,frame_id=piece
            Bottom.append(bottom_right)
    y_info=[i[1] for i in Bottom]
    y_median = np.median(y_info).astype(int)
    return y_median

In [ ]:
median_bottom(video_info)

In [ ]:
print(y_mean,y_median)

In [ ]:
test_imgs = put_points(frames,points_video,fps)

In [ ]:
show_frames(test_imgs,fps)

In [ ]:
save_video("test_imgs.mp4",test_imgs,fps,size)

In [ ]:
np.median([1,2,3,4,5,62,23,12,3]).astype(int)

In [ ]:
fps

In [ ]:
cv2.destroyAllWindows()

In [ ]:
cv2.line(img, (0,d),(size[0],d),(0, 255, 255))

In [ ]:
put_points(frames,points_video,fps)

In [ ]:
cv2.imshow("s",frames[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
templates = load_templates()

In [ ]:
video_path = "LAS/002_RI_full.mp4"

frames,fps,size = load_video_as_frames(video_path)

points_video = init_points(frames)

In [ ]:
video_info = get_video_info(frames,templates)
video_info

In [ ]:
video_info_thr = [[temp for temp in frame_info if temp[3]>0.9] for frame_info in video_info if [temp for temp in frame_info if temp[3]>0.9]]

In [ ]:
med = median_bottom(video_info_thr)

imgs = put_points(frames,points_video,fps,med)

In [ ]:
save_video("thr9.mp4",imgs,fps,size)

In [ ]:
test_imgs=frames.copy()
for frame_info in video_info:
    frame_id = frame_info[0][4]
    test_imgs[frame_id] = draw_templates(frames[frame_id],frame_info)

In [ ]:
med

In [ ]:
show_frames(test_imgs,fps)

In [ ]:
size[::-1]

In [ ]:
for frame in test_imgs:
    if frame.shape!=size[::-1]:
        print(True)
        break

In [ ]:
save_video("t002_RI_full.mp4",test_imgs,fps,size)

In [ ]:
def make(path):
    templates = load_templates()
    for videoname in os.listdir(path):
        if ".mp4" not in videoname:
            continue
            
        video_path = path+videoname
        print(video_path)
        
        frames,fps,size = load_video_as_frames(video_path)
        
        points_video = init_points(frames)
        
        video_info = get_video_info(frames,templates)
        
        med = median_bottom(video_info)
        
        imgs = put_points(frames,points_video,fps,med)
        
        dst_path = path+"med_ribs/"
        if not os.path.exists(dst_path):
            os.mkdir(dst_path)
            
        dst_name = dst_path+videoname
        save_video(dst_name,imgs,fps,size)
        print("Successfully saved ",dst_name)

In [ ]:
make("LAS/")

In [ ]:
from methods import *

In [ ]:
points_video[:3]

In [ ]:
def put_points(video,points_video,fps,median_bottom):
    
    imgs=[cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in video]
    ##imgs[img_id][point_id] : [x,y,confidence,frame_id]
    for img_id in range(len(imgs)):
        for x in range(10):
            if points_video[img_id][x][2]>1:
                color = (0,0,255) 
                cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),meidan_bottom), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)
            color = (0,255,0)
            cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id][x][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)

        cv2.line(imgs[img_id], (0,median_bottom),(imgs[img_id].shape[1],median_bottom),(0, 255, 255))
    
    return imgs

In [11]:
# points_video, video_info, frames, median_bottom

def points_error_correct(points_video,median_bottom,scale=None):

    mD = median_D(points_video,median_bottom)
    if scale:
        mD = scale*mD
    
    points_correct = points_video.copy()
    
    for frame_id in range(len(points_correct)):
        for point_id in range(len(points_correct[frame_id])):
            D=np.abs(points_correct[frame_id][point_id][1]-median_bottom)
            if D>mD:
                points_correct[frame_id][point_id][2]=1.1
    return points_correct

In [ ]:
points_correct = points_error_correct(points_video,median_bottom)

In [ ]:
imgs_correct = put_points(frames,points_correct,fps,median_bottom)

In [ ]:
show_frames(imgs_correct,fps)

In [9]:
def median_D(points_video,median):
    data = []
    for points_frame in points_video:
        for point in points_frame:
            data.append(np.abs(point[1]-median))
    med = np.median(data).astype(int)
    return med

In [ ]:
mmedian_D(points_video,med)

In [ ]:
D=dict()
for i in data:
    if i not in D:
        D[i]=1
    else:
        D[i]+=1
    

In [ ]:
DL=[[i,D[i]] for i in D]
DL.sort()
DL

In [ ]:
plt.plot([i[0] for i in DL],[i[1] for i in DL])
plt.show()

In [ ]:
def median_bottom(info):
    Bottom=[]
    for frame_info in info:
        for piece in frame_info:
            top_left,bottom_right,temp_id,confidence,frame_id=piece
            Bottom.append(bottom_right[1])
    y_median = np.median(Bottom).astype(int)
    return y_median

In [ ]:
video_info

In [15]:
def make(path):
    templates = load_templates()
    
    for videoname in os.listdir(path):
        if ".mp4" not in videoname:
            continue

        video_path = path+videoname
        print(video_path)

        frames,fps,size = load_video_as_frames(video_path)

        points_video = init_points(frames)

        video_info = get_video_info(frames,templates)

        mb = median_bottom(video_info)

        points_correct = points_error_correct(points_video,mb)

        imgs_correct = put_points(frames,points_correct,fps,mb)

        for i in range(len(video_info)):
            #video_info:[top_left,bottom_right,temp_id,confidence,i]#video_info:[top_left,bottom_right,temp_id,confidence,i]
            frame_id = video_info[i][0][4] 
            imgs_correct[frame_id] = draw_templates(imgs_correct[frame_id],video_info[i])

        dst_path = path+"corrected1/"
        if not os.path.exists(dst_path):
            os.mkdir(dst_path)

#         time = "".join([str(time.localtime(time.time())[i])+"-" for i in range(1,5)])#'3-3-20-21'
        dst_name = dst_path+videoname
        save_video(dst_name,imgs_correct,fps,size)
        print("Successfully saved ",dst_name)

In [16]:
make("LAS/")

LAS/001_RI_full.mp4
Successfully saved  LAS/corrected1/001_RI_full.mp4
LAS/002_RI_full.mp4
Successfully saved  LAS/corrected1/002_RI_full.mp4
LAS/003_RI_full.mp4
Successfully saved  LAS/corrected1/003_RI_full.mp4
LAS/004_RI_full.mp4
Successfully saved  LAS/corrected1/004_RI_full.mp4
LAS/005_RI_full.mp4
Successfully saved  LAS/corrected1/005_RI_full.mp4
LAS/006_RI_full.mp4
Successfully saved  LAS/corrected1/006_RI_full.mp4
LAS/007_RI_full.mp4
Successfully saved  LAS/corrected1/007_RI_full.mp4
LAS/008_RI_full.mp4
Successfully saved  LAS/corrected1/008_RI_full.mp4
LAS/009_RI_full.mp4
Successfully saved  LAS/corrected1/009_RI_full.mp4
LAS/010_RI_full.mp4
Successfully saved  LAS/corrected1/010_RI_full.mp4


In [5]:
templates = load_templates()

In [4]:
path = "test/"

for videoname in os.listdir(path):
    if ".mp4" not in videoname:
        continue

    video_path = path+videoname
    print(video_path)

    frames,fps,size = load_video_as_frames(video_path)

    points_video = init_points(frames)

    video_info = get_video_info(frames,templates)

    mb = median_bottom(video_info)

    points_correct = points_error_correct(points_video,mb)

    imgs_correct = put_points(frames,points_correct,fps,mb)
    
    #video_info:[top_left,bottom_right,temp_id,confidence,i]
    
    for i in range(len(video_info)):
        frame_id = video_info[i][0][4]
        imgs_correct[frame_id] = draw_templates(imgs_correct[frame_id],video_info[i])

    dst_path = path+"corrected1/"
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    dst_name = dst_path+videoname
    save_video(dst_name,imgs_correct,fps,size)
    print("Successfully saved ",dst_name)

test/001_RI_full.mp4


NameError: name 'templates' is not defined

In [14]:
points_correct = points_error_correct(points_video,mb)

imgs_correct = put_points(frames,points_correct,fps,mb)

#video_info:[top_left,bottom_right,temp_id,confidence,i]

for i in range(len(video_info)):
    frame_id = video_info[i][0][4]
    imgs_correct[frame_id] = draw_templates(imgs_correct[frame_id],video_info[i])

dst_path = path+"corrected1/"
if not os.path.exists(dst_path):
    os.mkdir(dst_path)

dst_name = dst_path+videoname
save_video(dst_name,imgs_correct,fps,size)
print("Successfully saved ",dst_name)

Successfully saved  test/corrected1/001_RI_full.mp4


In [ ]:
path = "test/"
for videoname in os.listdir(path):
    if ".mp4" not in videoname:
        continue

    vn = videoname

points_correct = points_error_correct(points_video,med)

imgs_correct = put_points(frames,points_correct,fps,med)

#video_info:[top_left,bottom_right,temp_id,confidence,i]

for i in range(len(video_info)):
    frame_id = video_info[i][0][4]
    imgs_correct[frame_id] = draw_templates(imgs_correct[frame_id],video_info[i])

dst_path = path+"corrected1/"
if not os.path.exists(dst_path):
    os.mkdir(dst_path)

dst_name = dst_path+vn
save_video(dst_name,imgs_correct,fps,size)
print("Successfully saved ",dst_name)

In [13]:
def put_points(video,points_video,fps,median_bottom):
    
    imgs=[cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in video]
    ##imgs[img_id][point_id] : [x,y,confidence,frame_id]
    for img_id in range(len(imgs)):
        for x in range(10):
            if points_video[img_id][x][2]>1:
                color = (0,0,255) 
                cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),median_bottom), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)
            color = (0,255,0)
            cv2.putText(imgs[img_id], 'o', (int(points_video[img_id][x][0]),int(points_video[img_id][x][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1)

        cv2.line(imgs[img_id], (0,median_bottom),(imgs[img_id].shape[1],median_bottom),(0, 255, 255))
    
    return imgs

In [ ]:
dst_path = path+"corrected/"
if not os.path.exists(dst_path):
    os.mkdir(dst_path)

dst_name = dst_path+videoname
save_video(dst_name,imgs_correct,fps,size)
print("Successfully saved ",dst_name)

In [ ]:
print(len(imgs_correct),len(video_info))

In [ ]:
len(frames)

In [ ]:
video_info[0]

In [ ]:
def draw_templates(frame,info,thr=0):
    img=frame.copy()
    for temp in info:
        top_left,bottom_right,temp_id,confidence,frame_id=temp
        if confidence>thr:
            cv2.rectangle(img, top_left, bottom_right, 255, 1)
            cv2.putText(img,"template:"+str(temp_id)+"  Confidence:"+str(round(confidence,2)),(top_left[0],top_left[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 1)
    return img

In [ ]:
for i in range(len(video_info)):
    frame_id = video_info[i][0][4]
    imgs_correct[frame_id] = draw_templates(imgs_correct[frame_id],video_info[i])

In [ ]:
show_frames(imgs_correct)


In [1]:
from methods import *

In [ ]:
#slide window